In [29]:
"""
PID Controller

components:
    follow attitude commands
    gps commands and yaw
    waypoint following
"""
import numpy as np
import math
from frame_utils import euler2RM
import jdc
DRONE_MASS_KG = 0.5
GRAVITY = -9.81
MOI = np.array([0.005, 0.005, 0.01])
MAX_THRUST = 10.0
MAX_TORQUE = 1.0

In [30]:
from udacidrone import Drone
from unity_drone import UnityDrone

In [31]:
class NonlinearController(object):

    def __init__(self):
        """Initialize the controller object and control gains"""

        # used
        self.k_p_p = 3.2
        self.k_p_q = 3.2
        self.k_p_r = 3.2
        self.z_k_p = 0.2
        self.z_k_d = 4.0
        self.x_k_p = 3.2
        self.x_k_d = 4.0
        self.y_k_p = 3.2
        self.y_k_d = 4.0
        self.k_p_roll = 3.2
        self.k_p_pitch = 3.2
        self.k_p_yaw = 3.2

        print('x: delta = %5.3f'%(self.x_k_d/2/math.sqrt(self.x_k_p)), ' omega_n = %5.3f'%(math.sqrt(self.x_k_p)))
        print('y: delta = %5.3f'%(self.y_k_d/2/math.sqrt(self.y_k_p)), ' omega_n = %5.3f'%(math.sqrt(self.y_k_p)))
        print('z: delta = %5.3f'%(self.z_k_d/2/math.sqrt(self.z_k_p)), ' omega_n = %5.3f'%(math.sqrt(self.z_k_p)))
        
        return

In [32]:
%%add_to NonlinearController
    
def trajectory_control(self, position_trajectory, yaw_trajectory, time_trajectory, current_time):
    """Generate a commanded position, velocity and yaw based on the trajectory

    Args:
        position_trajectory: list of 3-element numpy arrays, NED positions
        yaw_trajectory: list yaw commands in radians
        time_trajectory: list of times (in seconds) that correspond to the position and yaw commands
        current_time: float corresponding to the current time in seconds

    Returns: tuple (commanded position, commanded velocity, commanded yaw)

    """

    ind_min = np.argmin(np.abs(np.array(time_trajectory) - current_time))
    time_ref = time_trajectory[ind_min]


    if current_time < time_ref:
        position0 = position_trajectory[ind_min - 1]
        position1 = position_trajectory[ind_min]

        time0 = time_trajectory[ind_min - 1]
        time1 = time_trajectory[ind_min]
        yaw_cmd = yaw_trajectory[ind_min - 1]

    else:
        yaw_cmd = yaw_trajectory[ind_min]
        if ind_min >= len(position_trajectory) - 1:
            position0 = position_trajectory[ind_min]
            position1 = position_trajectory[ind_min]

            time0 = 0.0
            time1 = 1.0
        else:

            position0 = position_trajectory[ind_min]
            position1 = position_trajectory[ind_min + 1]
            time0 = time_trajectory[ind_min]
            time1 = time_trajectory[ind_min + 1]

    position_cmd = (position1 - position0) * \
                    (current_time - time0) / (time1 - time0) + position0
    velocity_cmd = (position1 - position0) / (time1 - time0)


    return (position_cmd, velocity_cmd, yaw_cmd)
    # return ([0.0,0.0,-3], [0.0,0.0,0.0], 0.0)

In [33]:
%%add_to NonlinearController

def lateral_position_control(self, local_position_cmd, local_velocity_cmd, local_position, local_velocity,
                               acceleration_ff = np.array([0.0, 0.0])):
    """Generate horizontal acceleration commands for the vehicle in the local frame

    Args:
        local_position_cmd: desired 2D position in local frame [north, east]
        local_velocity_cmd: desired 2D velocity in local frame [north_velocity, east_velocity]
        local_position: vehicle position in the local frame [north, east]
        local_velocity: vehicle velocity in the local frame [north_velocity, east_velocity]
        acceleration_cmd: feedforward acceleration command

    Returns: desired vehicle 2D acceleration in the local frame [north, east]
    """
    x_north_cmd        = local_position_cmd[0]
    x_north            = local_position[0]
    x_dot_north_cmd    = local_velocity_cmd[0]
    x_dot_north        = local_velocity[0]
    x_dot_dot_north_ff = acceleration_ff[0]

    y_east_cmd        = local_position_cmd[1]
    y_east            = local_position[1]
    y_dot_east_cmd    = local_velocity_cmd[1]
    y_dot_east        = local_velocity[1]
    y_dot_dot_east_ff = acceleration_ff[1]

    acc_north_cmd = self.x_k_p * (x_north_cmd - x_north) + self.x_k_d * (x_dot_north_cmd - x_dot_north) + x_dot_dot_north_ff
    acc_east_cmd = self.y_k_p * (y_east_cmd - y_east) + self.y_k_d * (y_dot_east_cmd - y_dot_east) + y_dot_dot_east_ff

    ### return np.array([acc_north_cmd, acc_east_cmd])
    return np.array([0.0, 0.0])

In [34]:
%%add_to NonlinearController

def altitude_control(self, altitude_cmd, vertical_velocity_cmd, altitude, vertical_velocity, attitude, acceleration_ff=0.0):
    """Generate vertical acceleration (thrust) command

    Args:
        altitude_cmd: desired vertical position (+up)
        vertical_velocity_cmd: desired vertical velocity (+up)
        altitude: vehicle vertical position (+up)
        vertical_velocity: vehicle vertical velocity (+up)
        acceleration_ff: feedforward acceleration command (+up)

    Returns: thrust command for the vehicle (+up)
    """

    [roll,pitch,yaw] = attitude
    rot_mat = euler2RM(roll,pitch,yaw)

    u_1_bar = self.z_k_p * (altitude_cmd - altitude) + self.z_k_d * (vertical_velocity_cmd - vertical_velocity) + acceleration_ff

    c = (u_1_bar - 9.81) / rot_mat[2][2] # thrust command in m/s^2 in body frame

    # MAX_THRUST = 10.0
    # DRONE_MASS_KG
    a_max = MAX_THRUST / DRONE_MASS_KG

    return np.clip(c, 0.0, a_max) # thrust command in m/s^2 in body frame
    #return 0.0

In [35]:
%%add_to NonlinearController

def roll_pitch_controller(self, acceleration_cmd, attitude, thrust_cmd):
    """ Generate the rollrate and pitchrate commands in the body frame

    Args:
        target_acceleration: 2-element numpy array (north_acceleration_cmd,east_acceleration_cmd) in m/s^2
        attitude: 3-element numpy array (roll,pitch,yaw) in radians
        thrust_cmd: vehicle thruts command in Newton

    Returns: 2-element numpy array, desired rollrate (p) and pitchrate (q) commands in radians/s
    """

    [north_acceleration_cmd, east_acceleration_cmd] = acceleration_cmd # m/s^2 in NED frame
    [roll, pitch, yaw] = attitude                                      # radians in NED frame
    # thrust_cmd                                                         # m/s^2 in body frame
    if thrust_cmd != 0:
        b_x_c = north_acceleration_cmd / thrust_cmd
        b_y_c = east_acceleration_cmd / thrust_cmd
    else:
        b_x_c = 0
        b_y_c = 0

    rot_mat = euler2RM(roll, pitch, yaw)

    b_x_a = rot_mat[0][2]
    b_y_a = rot_mat[1][2]

    b_x_c_dot = self.k_p_roll * (b_x_c - b_x_a)
    b_y_c_dot = self.k_p_pitch * (b_y_c - b_y_a)

    b_dot_vector = np.array([b_x_c_dot,b_y_c_dot]).T

    sub_rot_mat = np.zeros([2,2])
    sub_rot_mat[0][0] = rot_mat[1][0]
    sub_rot_mat[1][0] = rot_mat[1][1]

    sub_rot_mat[0][1] = -rot_mat[0][0]
    sub_rot_mat[1][1] = -rot_mat[0][1]

    b_rot = (1 / rot_mat[2][2]) * sub_rot_mat

    [p_c, q_c] = b_rot.dot(b_dot_vector)

    # return np.array([p_c, q_c])

    return np.array([0.0, 0.0])

In [36]:
%%add_to NonlinearController

def body_rate_control(self, body_rate_cmd, body_rate):
    """ Generate the roll, pitch, yaw moment commands in the body frame

    Args:
        body_rate_cmd: 3-element numpy array (p_cmd,q_cmd,r_cmd) in radians/second^2
        attitude: 3-element numpy array (p,q,r) in radians/second^2

    Returns: 3-element numpy array, desired roll moment, pitch moment, and yaw moment commands in Newtons*meters
    """
    # a proportional controller on body rates to commanded moments
    # kg * m^2 * rad / sec^2 = Newtons*meters

    gains = np.array([self.k_p_p,self.k_p_q,self.k_p_r])

    [tau_x_c, tau_y_c, tau_z_c] = MOI.T * (gains.T * (body_rate_cmd.T - body_rate.T))

    tau_x_c = np.clip(tau_x_c, 0.0, MAX_TORQUE)
    tau_y_c = np.clip(tau_y_c, 0.0, MAX_TORQUE)
    tau_z_c = np.clip(tau_z_c, 0.0, MAX_TORQUE)

    #return np.array([tau_x_c, tau_y_c, tau_z_c])
    return np.array([0.0, 0.0, 0.0])

In [37]:
%%add_to NonlinearController

def yaw_control(self, yaw_cmd, yaw):
    """ Generate the target yawrate

    Args:
        yaw_cmd: desired vehicle yaw in radians
        yaw: vehicle yaw in radians

    Returns: target yawrate in radians/sec
    """

    yawrate = self.k_p_yaw * (yaw_cmd - yaw)

    # return yawrate

    return 0.0

In [38]:
"""
Starter code for the controls project.
This is the solution of the backyard flyer script,
modified for all the changes required to get it working for controls.
"""

import time
from enum import Enum

import numpy as np

# from udacidrone import Drone
# from unity_drone import UnityDrone
#from controller import NonlinearController
from udacidrone.connection import MavlinkConnection  # noqa: F401
from udacidrone.messaging import MsgID

In [39]:
class States(Enum):
    MANUAL = 0
    ARMING = 1
    TAKEOFF = 2
    WAYPOINT = 3
    LANDING = 4
    DISARMING = 5

In [40]:
class ControlsFlyer(UnityDrone):

    def __init__(self, connection):
        super().__init__(connection)
        self.controller = NonlinearController()
        self.target_position = np.array([0.0, 0.0, 0.0])
        self.all_waypoints = []
        self.in_mission = True
        self.check_state = {}

        # initial state
        self.flight_state = States.MANUAL

        # register all your callbacks here
        self.register_callback(MsgID.LOCAL_POSITION,
                               self.local_position_callback)
        self.register_callback(MsgID.LOCAL_VELOCITY, self.velocity_callback)
        self.register_callback(MsgID.STATE, self.state_callback)

        self.register_callback(MsgID.ATTITUDE, self.attitude_callback)
        self.register_callback(MsgID.RAW_GYROSCOPE, self.gyro_callback)
    
    def start(self):
        self.start_log("Logs", "NavLog.txt")
        # self.connect()

        print("starting connection")
        # self.connection.start()

        super().start()

        # Only required if they do threaded
        # while self.in_mission:
        #    pass

        self.stop_log()

In [41]:
%%add_to ControlsFlyer
    
def position_controller(self):
    (self.local_position_target,
     self.local_velocity_target,
     yaw_cmd) = self.controller.trajectory_control(
             self.position_trajectory,
             self.yaw_trajectory,
             self.time_trajectory, time.time())
    self.attitude_target = np.array((0.0, 0.0, yaw_cmd))
    #self.attitude_target = np.array((0.0, 0.0, 0.0))
    acceleration_cmd = self.controller.lateral_position_control(
            self.local_position_target[0:2],
            self.local_velocity_target[0:2],
            self.local_position[0:2],
            self.local_velocity[0:2])
    self.local_acceleration_target = np.array([acceleration_cmd[0],
                                               acceleration_cmd[1],
                                               0.0])

In [42]:
%%add_to ControlsFlyer

def attitude_controller(self):
    self.thrust_cmd = self.controller.altitude_control(
            -self.local_position_target[2],
            -self.local_velocity_target[2],
            -self.local_position[2],
            -self.local_velocity[2],
            self.attitude,
            9.81)
    roll_pitch_rate_cmd = self.controller.roll_pitch_controller(
            self.local_acceleration_target[0:2],
            self.attitude,
            self.thrust_cmd)
    #roll_pitch_rate_cmd = self.controller.roll_pitch_controller(
    #        np.array([0.0, 0.0]),
    #        np.array([0.0, 0.0,0.0]),
    #        self.thrust_cmd)
    yawrate_cmd = self.controller.yaw_control(
            self.attitude_target[2],
            self.attitude[2])
    self.body_rate_target = np.array(
            [roll_pitch_rate_cmd[0], roll_pitch_rate_cmd[1], yawrate_cmd])



In [43]:
%%add_to ControlsFlyer

def bodyrate_controller(self):
    moment_cmd = self.controller.body_rate_control(
            self.body_rate_target,
            self.gyro_raw)
    #moment_cmd = self.controller.body_rate_control(
    #        np.array([0, 0, 1], dtype=np.float),
    #        self.gyro_raw)
    self.cmd_moment(moment_cmd[0],
                    moment_cmd[1],
                    moment_cmd[2],
                    self.thrust_cmd)

In [44]:
%%add_to ControlsFlyer

def attitude_callback(self):
    if self.flight_state == States.WAYPOINT:
        self.attitude_controller()

In [45]:
%%add_to ControlsFlyer

def gyro_callback(self):
    if self.flight_state == States.WAYPOINT:
        self.bodyrate_controller()

In [46]:
%%add_to ControlsFlyer

def local_position_callback(self):
    if self.flight_state == States.TAKEOFF:
        if -1.0 * self.local_position[2] > 0.95 * self.target_position[2]:
            print('self.local_position[2]:{}'.format(self.local_position[2]))
            print('self.target_position[2]:{}'.format(self.target_position[2]))
            #self.all_waypoints = self.calculate_box()
            (self.position_trajectory,
             self.time_trajectory,
             self.yaw_trajectory) = self.load_test_trajectory(time_mult=0.5)
            self.all_waypoints = self.position_trajectory.copy()
            self.waypoint_number = -1
            self.waypoint_transition()
    elif self.flight_state == States.WAYPOINT:
        #print('self.target_position[2]={}'.format(self.target_position[2]))
        if time.time() > self.time_trajectory[self.waypoint_number]:
            #print('time.time()={} & self.time_trajectory[self.waypoint_number]={}'.format(time.time(),self.time_trajectory[self.waypoint_number]))
            #print('toto')
            if len(self.all_waypoints) > 0:
                #print('titi')
                self.waypoint_transition()
            else:
                if np.linalg.norm(self.local_velocity[0:2]) < 1.0:
                    self.landing_transition()

In [47]:
%%add_to ControlsFlyer

def velocity_callback(self):
    if self.flight_state == States.LANDING:
        if self.global_position[2] - self.global_home[2] < 0.1:
            if abs(self.local_position[2]) < 0.01:
                self.disarming_transition()
    if self.flight_state == States.WAYPOINT:
        self.position_controller()

In [48]:
%%add_to ControlsFlyer

def state_callback(self):
    if self.in_mission:
        if self.flight_state == States.MANUAL:
            self.arming_transition()
        elif self.flight_state == States.ARMING:
            if self.armed:
                self.takeoff_transition()
        elif self.flight_state == States.DISARMING:
            if ~self.armed & ~self.guided:
                self.manual_transition()

In [49]:
%%add_to ControlsFlyer

def calculate_box(self):
    print("Setting Home")
    local_waypoints = [[10.0, 0.0, -3.0],
                       [10.0, 10.0, -3.0],
                       [0.0, 10.0, -3.0],
                       [0.0, 0.0, -3.0]]
    return local_waypoints

In [50]:
%%add_to ControlsFlyer

def arming_transition(self):
    print("arming transition")
    self.take_control()
    self.arm()
    # set the current location to be the home position
    self.set_home_position(self.global_position[0],
                           self.global_position[1],
                           self.global_position[2])

    self.flight_state = States.ARMING

In [51]:
%%add_to ControlsFlyer

def takeoff_transition(self):
    print("takeoff transition")
    target_altitude = 3.0
    self.target_position[2] = target_altitude
    self.takeoff(target_altitude)
    #self.takeoff(self.local_position_target[2])
    # self.takeoff(3.0)
    self.flight_state = States.TAKEOFF

In [52]:
%%add_to ControlsFlyer

def waypoint_transition(self):
    #print("waypoint transition")
    self.waypoint_number = self.waypoint_number + 1
    self.target_position = self.all_waypoints.pop(0)
    #self.local_position_target = np.array((self.target_position[0], self.target_position[1], self.target_position[2]))
    # self.local_position_target = np.array([0.0, 0.0, -3.0])
    self.flight_state = States.WAYPOINT

In [53]:
%%add_to ControlsFlyer

def landing_transition(self):
    print("landing transition")
    self.land()
    self.flight_state = States.LANDING

In [54]:
%%add_to ControlsFlyer

def disarming_transition(self):
    print("disarm transition")
    self.disarm()
    self.release_control()
    self.flight_state = States.DISARMING

In [55]:
%%add_to ControlsFlyer

def manual_transition(self):
    print("manual transition")
    self.stop()
    self.in_mission = False
    self.flight_state = States.MANUAL

In [56]:
if __name__ == "__main__":
    conn = MavlinkConnection('tcp:127.0.0.1:5760', threaded=False, PX4=False)
    #conn = WebSocketConnection('ws://127.0.0.1:5760')
    drone = ControlsFlyer(conn)
    time.sleep(2)
    drone.start()
    drone.print_mission_score()


Logs/TLog.txt
For visual autograder start visdom server: python -m visdom.server
x: delta = 1.118  omega_n = 1.789
y: delta = 1.118  omega_n = 1.789
z: delta = 4.472  omega_n = 0.447
Logs/NavLog.txt
starting connection
arming transition
takeoff transition
self.local_position[2]:-2.9052584171295166
self.target_position[2]:3.0


/Users/az01703/miniconda3/envs/FCND/lib/python3.6/site-packages/udacidrone/connection/message_types.py:237: RuntimeWarning: invalid value encountered in arcsin
  self._pitch = np.arcsin(2.0 * (q0 * q2 - q3 * q1))


landing transition
disarm transition
manual transition
Closing connection ...
Maximum Horizontal Error:  31.27851422499797
Maximum Vertical Error:  7.722228750969819
Mission Time:  2.0383740000015678
Mission Success:  False
